In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obstructive_sleep_apnea"
cohort = "GSE49800"

# Input paths
in_trait_dir = "../../input/GEO/Obstructive_sleep_apnea"
in_cohort_dir = "../../input/GEO/Obstructive_sleep_apnea/GSE49800"

# Output paths
out_data_file = "../../output/preprocess/Obstructive_sleep_apnea/GSE49800.csv"
out_gene_data_file = "../../output/preprocess/Obstructive_sleep_apnea/gene_data/GSE49800.csv"
out_clinical_data_file = "../../output/preprocess/Obstructive_sleep_apnea/clinical_data/GSE49800.csv"
json_path = "../../output/preprocess/Obstructive_sleep_apnea/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any, Union

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from leukocytes
# which was hybridized to Affymetrix Genechip Human Gene 1.0 ST microarrays
is_gene_available = True

# 2.1 Data Availability and 2.2 Data Type Conversion
# For trait: The data shows comparison between baseline (no treatment) and CPAP treatment
# This is indicated in row 1 of the sample characteristics
trait_row = 1

def convert_trait(value: str) -> Optional[int]:
    """Convert OSA treatment information to binary values.
    0: Baseline/no treatment
    1: CPAP treatment
    """
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    if 'none' in value or 'baseline' in value:
        return 0
    elif 'cpap' in value:
        return 1
    else:
        return None

# For age: There's no age information available in the sample characteristics
age_row = None

def convert_age(value: str) -> Optional[float]:
    """Convert age information to float values.
    Since age data is not available, this function is a placeholder.
    """
    return None

# For gender: There's no gender information available in the sample characteristics
gender_row = None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender information to binary values.
    Since gender data is not available, this function is a placeholder.
    """
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Create a clinical data DataFrame from the sample characteristics dictionary
    sample_chars = {
        0: ['subject: 1', 'subject: 2', 'subject: 3', 'subject: 4', 'subject: 5', 'subject: 6', 'subject: 7', 
            'subject: 8', 'subject: 9', 'subject: 10', 'subject: 11', 'subject: 12', 'subject: 13', 'subject: 14', 
            'subject: 15', 'subject: 16', 'subject: 17', 'subject: 18'],
        1: ['treatment: none, baseline', 'treatment: CPAP']
    }
    
    # Create the clinical DataFrame with proper structure
    # Each subject has both baseline and CPAP measurements (total of 36 samples)
    data = {}
    
    for subject in sample_chars[0]:
        subject_num = subject.split(": ")[1]
        
        # For baseline treatment
        sample_id = f"GSM_{subject_num}_baseline"
        data[sample_id] = ["treatment: none, baseline"]
        
        # For CPAP treatment
        sample_id = f"GSM_{subject_num}_CPAP" 
        data[sample_id] = ["treatment: CPAP"]
    
    # Create the clinical DataFrame with proper structure
    clinical_data = pd.DataFrame(data)
    clinical_data.index = [trait_row]  # Set index to trait_row (1)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the selected clinical features to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 4: Gene Identifier Review

In [ ]:
# Review the gene identifiers
# These appear to be probe IDs (numeric identifiers) from a microarray platform, not human gene symbols
# They are likely Illumina BeadChip probe IDs which need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for probe IDs and gene symbols in the gene annotation data
# Looking at the data, 'ID' contains the probe identifiers (matching the gene expression data index)
# The 'gene_assignment' column contains gene symbol information

# Create gene mapping dataframe with probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print information about the mapping dataframe
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 2. Apply the gene mapping to convert probe-level measurements to gene expression data
# This will split probe values equally among mapped genes and sum values for each gene
gene_expr_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols (standardize and handle synonyms)
gene_expr_data = normalize_gene_symbols_in_index(gene_expr_data)

# Print information about the gene expression data
print(f"\nProcessed gene expression data shape: {gene_expr_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_expr_data.index[:5])

# Store the processed gene expression data in gene_data
gene_data = gene_expr_data

# 3. Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Properly reference the gene expression data from previous steps
# In the previous step we created gene_expr_data from the mapping operation
# We need to properly reference this variable
gene_data = gene_expr_data

# Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data using the correct row indices
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background information and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Print clinical data structure to understand available rows
print("Clinical data index:", clinical_data.index.tolist())
print("Sample characteristics:")
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print(sample_characteristics_dict)

# Based on the sample characteristics, we only have 'treatment' information
# No age or gender information is available
trait_row = 1  # Treatment row (baseline vs CPAP)
age_row = None  # No age information available
gender_row = None  # No gender information available

def convert_trait(value):
    """Convert OSA treatment information to binary values.
    0: Baseline/no treatment
    1: CPAP treatment
    """
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    if 'none' in value or 'baseline' in value:
        return 0
    elif 'cpap' in value:
        return 1
    else:
        return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=None
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait is biased - instead of using an undefined function, 
# directly use the appropriate judge function
print("\nChecking for bias in the trait variable:")
trait_type = 'binary' if len(linked_data[trait].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_biased = judge_continuous_variable_biased(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset compares gene expression before (baseline) and after CPAP therapy in patients with severe OSA."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Re-extract the gene expression data to ensure we have it properly defined
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Extract gene annotation data for mapping
gene_annotation = get_gene_annotation(soft_file)
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Based on the sample characteristics from previous steps, we only have treatment information
print("Clinical data structure:")
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print(sample_characteristics_dict)

# Use only the treatment information (row 1)
trait_row = 1  # treatment status (baseline vs CPAP)
age_row = None  # No age information available
gender_row = None  # No gender information available

def convert_trait(value):
    """
    Convert treatment information to binary:
    0: Baseline/no treatment
    1: CPAP treatment
    """
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    if 'none' in value or 'baseline' in value:
        return 0
    elif 'cpap' in value:
        return 1
    else:
        return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait is biased
print("\nChecking for bias in the trait variable:")
# The trait in this dataset is binary (baseline vs CPAP)
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset compares gene expression before (baseline) and after CPAP therapy in patients with severe OSA."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")